In [1]:
from torch_geometric.datasets import QM9


dataset = QM9(root='data/QM9')
print("Number of molecules:", len(dataset))

/usr/local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Number of molecules: 130831


In [8]:
from functions_to_abstract_data import extract_qm9_data
data_df = extract_qm9_data(dataset, n_samples=130000)
print(data_df)


Processed 1000/130000 molecules...
Processed 2000/130000 molecules...
Processed 3000/130000 molecules...
Processed 4000/130000 molecules...
Processed 5000/130000 molecules...
Processed 6000/130000 molecules...
Processed 7000/130000 molecules...
Processed 8000/130000 molecules...
Processed 9000/130000 molecules...
Processed 10000/130000 molecules...
Processed 11000/130000 molecules...
Processed 12000/130000 molecules...
Processed 13000/130000 molecules...
Processed 14000/130000 molecules...
Processed 15000/130000 molecules...
Processed 16000/130000 molecules...
Processed 17000/130000 molecules...
Processed 18000/130000 molecules...
Processed 19000/130000 molecules...
Processed 20000/130000 molecules...
Processed 21000/130000 molecules...
Processed 22000/130000 molecules...
Processed 23000/130000 molecules...
Processed 24000/130000 molecules...
Processed 25000/130000 molecules...
Processed 26000/130000 molecules...
Processed 27000/130000 molecules...
Processed 28000/130000 molecules...
P

In [9]:
print(data_df.isna().sum())

index        0
smiles       0
num_atoms    0
mu           0
alpha        0
HOMO         0
LUMO         0
gap          0
r2           0
zpve         0
U0           0
U298         0
H298         0
G298         0
Cv           0
U0_atom      0
U298_atom    0
H298_atom    0
G298_atom    0
A            0
B            0
C            0
atoms        0
pos          0
edge_idx     0
edge_attr    0
dtype: int64


In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score

# --- Step 1: Choose features and target ---
features = ['num_atoms','mu','alpha','r2','zpve','U0','U298','H298','G298','Cv']
X = data_df[features].values
y = data_df['gap'].values

# --- Step 2: Train/test split ---
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# --- Step 3: Scale features ---
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# --- Step 4: Train Linear Regression ---
model = LinearRegression()
model.fit(X_train_scaled, y_train)

# --- Step 5: Predict and evaluate ---
y_pred = model.predict(X_test_scaled)

mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Linear Regression MAE (eV): {mae:.3f}")
print(f"Linear Regression R²: {r2:.3f}")

# --- Step 6: Inspect coefficients ---
print("\nFeature coefficients:")
for feat, coef in zip(features, model.coef_):
    print(f"{feat}: {coef:.3f}")


Linear Regression MAE (eV): 0.704
Linear Regression R²: 0.537

Feature coefficients:
num_atoms: 2.259
mu: -0.062
alpha: -1.215
r2: 0.493
zpve: -1.237
U0: 57836.398
U298: -30137.366
H298: -23266.145
G298: -4432.127
Cv: 1.557
